### Project Files
- **snow.ipynb** — main analysis and design rationale  
- **cholera_interactive.html** — final interactive visualization (open in browser)  
- **data/** — source datasets (Snow’s data)  


In [13]:
import folium
from folium.plugins import HeatMap, MarkerCluster, MiniMap, Fullscreen
import pandas as pd

In [14]:
# --- Load and prepare data ---
deaths = pd.read_csv("data/Deaths.tsv", sep="\t")
pumps = pd.read_csv("data/Pumps.tsv", sep="\t")

deaths.columns = deaths.columns.str.lower()
pumps.columns = pumps.columns.str.lower()

# Convert Snow’s original (x, y) map coordinates to rough latitude/longitude
def to_latlon(x, y):
    lon = -0.137 + (x - deaths['x'].mean()) * 0.00003
    lat = 51.513 + (y - deaths['y'].mean()) * 0.00003
    return lat, lon

deaths[['lat', 'lon']] = deaths.apply(lambda r: to_latlon(r['x'], r['y']), axis=1, result_type='expand')
pumps[['lat', 'lon']] = pumps.apply(lambda r: to_latlon(r['x'], r['y']), axis=1, result_type='expand')

# --- Create map ---
m = folium.Map(location=[51.513, -0.137], zoom_start=16, tiles='CartoDB positron')

# --- Create feature layers ---
pump_layer = folium.FeatureGroup(name="Water Pumps")
heat_layer = folium.FeatureGroup(name="Cholera Deaths (Heatmap)")
cluster_layer = folium.FeatureGroup(name="Individual Deaths")

# Add pumps
for i, row in pumps.iterrows():
    folium.Marker(
        [row['lat'], row['lon']],
        icon=folium.Icon(color='blue', icon='tint', prefix='fa'),
        popup=f"<b>Pump {i+1}</b><br>Possible source of infection"
    ).add_to(pump_layer)

# Add heatmap
HeatMap(deaths[['lat', 'lon']], radius=8, blur=12, min_opacity=0.5).add_to(heat_layer)

# Optional: add individual death markers
marker_cluster = MarkerCluster().add_to(cluster_layer)
for _, row in deaths.iterrows():
    folium.CircleMarker(
        location=[row['lat'], row['lon']],
        radius=3,
        color='red',
        fill=True,
        fill_color='red',
        fill_opacity=0.6
    ).add_to(marker_cluster)

# --- Add layers to map ---
pump_layer.add_to(m)
heat_layer.add_to(m)
cluster_layer.add_to(m)

# --- Add interactive plugins ---
MiniMap(toggle_display=True).add_to(m)
Fullscreen().add_to(m)
folium.LayerControl(collapsed=False).add_to(m)

# --- Add title ---
title_html = '''
     <h3 align="center" style="font-size:20px">
     <b>John Snow's 1854 Cholera Map (Interactive Reimagining)</b></h3>
     <p align="center" style="font-size:14px">
     Explore the spatial distribution of cholera deaths and pump locations around Broad Street, Soho.
     </p>
     '''
m.get_root().html.add_child(folium.Element(title_html))

# --- Display or save ---
m.save("cholera_interactive.html")
m


In [12]:


# Load your data (TSV)
deaths = pd.read_csv("data/Deaths.tsv", sep="\t")
pumps = pd.read_csv("data/Pumps.tsv", sep="\t")

# Create the base map
m = folium.Map(location=[51.513, -0.137], zoom_start=17, tiles="CartoDB positron")

# Layer groups
heat_layer = folium.FeatureGroup(name="Cholera Death Heatmap")
pump_layer = folium.FeatureGroup(name="Water Pumps")
cluster_layer = folium.FeatureGroup(name="Death Cluster")

# 🧭 Add pumps
for i, row in pumps.iterrows():
    folium.Marker(
        [row['y'], row['x']],
        icon=folium.Icon(color='blue', icon='tint', prefix='fa'),
        popup=f"<b>Pump {i+1}</b><br>Possible source of infection"
    ).add_to(pump_layer)

# 🔥 Add death heatmap
HeatMap(deaths[['y', 'x']], radius=10, blur=15, min_opacity=0.5).add_to(heat_layer)

# 💀 Optional: clustered individual death markers
marker_cluster = MarkerCluster().add_to(cluster_layer)
for _, row in deaths.iterrows():
    folium.CircleMarker(
        location=[row['y'], row['x']],
        radius=3,
        color='red',
        fill=True,
        fill_color='red',
        fill_opacity=0.6
    ).add_to(marker_cluster)

# Add feature layers to map
pump_layer.add_to(m)
heat_layer.add_to(m)
cluster_layer.add_to(m)

# Add useful plugins
MiniMap(toggle_display=True).add_to(m)
Fullscreen().add_to(m)
folium.LayerControl(collapsed=False).add_to(m)

# 🧾 Add title (custom HTML)
title_html = '''
     <h3 align="center" style="font-size:20px">
     <b>John Snow's 1854 Cholera Outbreak Map (Interactive Reimagining)</b></h3>
     <p align="center" style="font-size:14px">Explore the pumps, deaths, and hotspots around Broad Street</p>
     '''
m.get_root().html.add_child(folium.Element(title_html))

# Save or show map
m.save("cholera_interactive.html")
m
